## Extract features

El proceso de cuantificar una imagen se llama extracción de características.

El proceso de extracción de características rige las reglas, los algoritmos y las metodologías que utilizamos para cuantificar de forma abstracta el contenido de una imagen utilizando solo una lista de números, denominada vector de características. Los descriptores de imagen y los descriptores de características gobiernan cómo se abstrae y cuantifica una imagen, mientras que los vectores de características son la salida de descriptores y se utilizan para cuantificar la imagen. En conjunto, este proceso se denomina extracción de características.

El proceso de extracción de características ha existido desde que existe la visión por ordenador. Los métodos han cambiado y hemos mejorado mucho en hacerlo, pero el proceso general de ingresar una imagen, ejecutar un algoritmo de extracción de características en la imagen y generar una lista de números para cuantificar la imagen sigue siendo el mismo. Casi cualquier algoritmo que intenta comprender e interpretar el contenido de una imagen utiliza la extracción de características en algún momento. Podríamos estar extrayendo características de una imagen por una variedad de razones; para comparar con imágenes por similitud; para clasificar las imágenes en los resultados de búsqueda al crear un motor de búsqueda de imágenes o para usar cuando se entrena un clasificador de imágenes para reconocer el contenido de una imagen.

Los vectores de características se utilizan para representar una variedad de propiedades de una imagen, como la forma, el color o la textura de un objeto en una imagen. También pueden combinar varias propiedades. Un vector de características podría representar conjuntamente forma y color. O podría representar textura y forma. O podría representar a los tres

## Image Descriptors

Un descriptor de imagen es un algoritmo y metodología que nos dice cómo se cuantifica globalmente una imagen de entrada y devuelve un vector de característica que representa de manera abstracta el contenido de la imagen.

El término clave que debemos entender aquí es global: esto implica que estamos examinando toda la imagen y utilizando toda la imagen en el cálculo de nuestro vector de características.

Los descriptores de imágenes cuantifican la forma, el color, la textura o cualquier combinación de los tres. Finalmente, la salida de nuestro descriptor de imagen es una lista de números: nuestro vector de características o simplemente características. Cuando se presenta con una imagen de entrada, nuestro descriptor de imagen devolverá 1 vector de características. Los ejemplos de descriptores de imagen incluyen estadísticas de canales de color, histogramas de color y patrones binarios locales...

Uno de los principales beneficios de los descriptores de imagen es que tienden a ser mucho más simples que los descriptores de características. Además, una vez extraídos, los vectores de características derivados de los descriptores de imagen se pueden pasar inmediatamente a otros métodos de visión por computadora, como crear un clasificador para reconocer el contenido de una imagen o crear un motor de búsqueda de imágenes. Sin embargo, esta simplicidad a menudo tiene un precio. Muchas veces, aunque son básicos y fáciles de usar, nuestros descriptores de imagen no son robustos a los cambios en la forma en que se gira, se traduce o cómo cambian los puntos de vista de una imagen. Si ese es el caso, muchas veces necesitaremos usar los descriptores de funciones más potentes.

¿Pero qué pasa si quisiéramos describir múltiples regiones de una imagen? Features descriptors que se explican en el otro notebook

# Computer Vision – Color channel Stadistics

El descriptor de imagen del canal de color se puede dividir en tres pasos: 

- Paso 1: Separar la imagen de entrada en sus canales respectivos. Para una imagen RGB, queremos examinar cada uno de los canales rojo, verde y azul de forma independiente. 

- Paso 2: Calcular varias estadísticas para cada canal, como la media, la desviación estándar, el sesgo y la curtosis. 

- Paso 3: Concatenar las estadísticas para formar una "lista" de estadísticas para cada canal de color: esto se convierte en nuestro vector de características. Por ejemplo, dada una imagen RGB de entrada, nuestra pipeline se vería así:

![](images/Imagen1.png)

A la izquierda tenemos nuestra imagen de entrada de la que queremos extraer estadísticas de canales de color. En el medio se encuentra nuestro descriptor de imagen: este cuadro nos dira cómo extraemos las estadísticas del canal de color. Y, finalmente, a la derecha tenemos nuestro vector de características de salida que consiste en la media y la desviación estándar para cada uno de los canales Rojo, Verde y Azul, respectivamente.

Calculamos la media y la desviación estándar de cada canal Rojo, Verde y Azul de nuestra imagen de entrada, concatenamos estas medias y desviaciones estándar juntas y utilizamos esta lista de estadísticas como nuestro vector de características. A partir de ahí, utilizamos la distancia euclidiana para comparar similitudes en las imágenes. Las imágenes con distancias más pequeñas se encuentran juntas en el espacio euclidiano y se consideran más similares. Las imágenes con distancias más grandes son por lo tanto consideradas menos similares.


### Setup

In [2]:
from scipy.spatial import distance as dist
from imutils import paths
import numpy as np
import cv2

In [3]:

# Tomaremos la lista de rutas de imagen e inicializaremos el índice para almacenar el nombre de archivo de la imagen
# y sus vector de características
imagePaths = sorted(list(paths.list_images("C:/Users/algonzalez/source/repos/Computer_Vision/2_Image_Descriptors/images/colo_channel_stats/trex/")))
index = {}

# loop sobre las rutas de los ficheros
for imagePath in imagePaths:
    # Cargar imagen y extraer nombre del fichero
    image = cv2.imread(imagePath)
    filename = imagePath[imagePath.rfind("/") + 1:]

    # extraer la media y la desviación estándar de cada canal del
    # Imagen BGR, luego actualizaremos el índice con el vector de características
    (means, stds) = cv2.meanStdDev(image)
    features = np.concatenate([means, stds]).flatten()
    index[filename] = features

# Muestra la imagen de consulta y agarra las claves ordenadas del diccionario de índice.
query = cv2.imread(imagePaths[0])
cv2.imshow("Query (trex_01.png)", query)
keys = sorted(index.keys())

for (i, k) in enumerate(keys):
    if k == "trex_01.png":
        continue
  
    # Cargar la imagen actual y calculeremos la distancia euclidiana entre la
    # imagen de consulta (es decir, la primera imagen) y la imagen actual
    image = cv2.imread(imagePaths[i])
    d = dist.euclidean(index["trex_01.png"], index[k])

    # Muestra la distancia entre la imagen de consulta y la imagen actual.
    cv2.putText(image, "%.2f" % (d), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
    cv2.imshow(k, image)

cv2.waitKey(0)
cv2.destroyAllWindows()